## Import Packages, Training Data Subset, and Intermediate Test Data

In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, RobustScaler
from sklearn.compose import ColumnTransformer 
from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import auc

%matplotlib inline

In [2]:
data_path = 'C:/Users/akipp/Documents/GitHub/Data/Insurance-Demo-Data/'
train_data_name = 'exercise_40_train.csv'

In [3]:
train_data = pd.read_csv(data_path + train_data_name)

## Train/Test Split and Data Cleaning

In [4]:
def data_cleaning(df):
    # x7 is a percentage amount which needs to be converted to a standard float value; contains some values in 
    # scientific notation which needs to be accounted for
    def percent_to_float(x):
        x_split = x.strip('%').split('e')
        if len(x_split) == 2:
            return(float(x_split[0]) * 10**float(x_split[1]))
        else:
            return(float(x_split[0]))

    df['x7'] = df.x7.apply(percent_to_float)


    # x19 is a dollar amount which needs to be 
    df['x19'] = df.x19.apply(lambda x: float(x.strip('$')))

    # Consolidate day of week names
    df['x3'] = df['x3'].map({'Monday': 'Monday', 'Tuesday': 'Tuesday', 'Wednesday': 'Wednesday', 'Thursday': 'Thursday', 
                             'Friday': 'Friday', 'Saturday': 'Saturday', 'Sunday': 'Sunday', 
                             'Mon': 'Monday', 'Tue': 'Tuesday', 'Wed': 'Wednesday', 'Thur': 'Thursday', 
                             'Fri': 'Friday', 'Sat': 'Saturday', 'Sun': 'Sunday'})
    # Replace missing values of what is probably a gender or sex column with 'other'
    df['x24'] = df.x24.fillna('other')
    # Replace missing values of what is probably a car make or brand column with 'other'
    df['x77'] = df.x77.fillna('other')
    # Replace missing values in x33, x99, and x79 with 'missing' because we don't know what is missing here;
    # Map the values in x79 to strings so it can having missing as a category and will work with OHE
    df['x99'] = df.x99.fillna('missing')
    df['x79'] = df['x79'].map({1.0: '1', 0.0: '0'}).fillna('missing')
    df['x33'] = df.x33.fillna('missing')
    # Drop columns x39 because it is not useful - only has a single value and no missing values.
    df.drop(columns = ['x39'], inplace = True)
    
    # Output initially cleaned data
    return(df)

In [5]:
# No issues with doing the data cleaning before train/test split because all transformations are element-wise only and there is no data leakage
train_data = data_cleaning(train_data)

In [6]:
y = train_data['y']
X = train_data.drop(columns = ['y'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, stratify = y, random_state = 42)

In [7]:
print(X_train.shape)
print(X_test.shape)

(36000, 99)
(4000, 99)


## Logistic Regression Modeling Pipeline

In [8]:
def lr_model_gscv(X_train, y_train, smote = True):
    # One-Hot Enconde categorical features
    cat_feats = X_train.columns[X_train.dtypes == 'object']
    
    # An imblearn pipeline is needed when using SMOTE in the pipeline because the function doesn't 
    # have a fit_transform method and can't be used with a sklearn pipeline. Also, the imblearn 
    # pipeline will ensure that SMOTE is only applied to training data, even in cross-validation.
    if smote == True: 
        pipeline = imbpipeline(steps = [('ohe', ColumnTransformer(transformers = [('ohe', OneHotEncoder(handle_unknown = 'ignore', sparse = False), cat_feats)], remainder = 'passthrough')),
                                        # Inputing missing continuous feature data with chained equations
                                        ('mice', IterativeImputer(max_iter = 20, tol = 0.01, n_nearest_features = 50, random_state = 42, verbose = 1)), 
                                        # Creating synthetic data from the minority class
                                        ('smote', SMOTE(k_neighbors = 10, random_state = 42)), 
                                        # Attempting to normalize the features with the yeo-johnson transformation (box-cox also an option, but requires all data to be positive)
                                        ('gaussian', PowerTransformer()),
                                        # Robust scaling of the data, rather than Standard scaling, because it is less impacted by outliers
                                        ('scaler', RobustScaler()),
                                        ('classifier', LogisticRegression(random_state = 42))
                                       ])
    # When not using SMOTE, a sklearn pipeline works fine
    else:
        pipeline = Pipeline(steps = [('ohe', ColumnTransformer(transformers = [('ohe', OneHotEncoder(handle_unknown = 'ignore', sparse = False), cat_feats)], remainder = 'passthrough')),
                                     # Inputing missing continuous feature data with chained equations
                                     ('mice', IterativeImputer(max_iter = 20, tol = 0.01, n_nearest_features = 50, random_state = 42, verbose = 1)),
                                     # Attempting to normalize the features with the yeo-johnson transformation (box-cox also an option, but requires all data to be positive)
                                     ('gaussian', PowerTransformer()),
                                     # Robust scaling of the data, rather than Standard scaling, because it is less impacted by outliers
                                     ('scaler', RobustScaler()),
                                     ('classifier', LogisticRegression(random_state = 42))
                                    ])
    # Specify how the folds should be created in GridSearchCV
    stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=42)
    # Define the parameter grid for GridSearchCV
    # defaults: 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100 (doesn't converge at 100), 'C': 1.0
    param_grid = {'classifier__C': [0.01, 0.1, 1, 10, 100], 'classifier__max_iter': [1000]} 
    grid_search = GridSearchCV(estimator = pipeline,
                               param_grid = param_grid,
                               scoring = 'roc_auc',
                               cv = stratified_kfold,
                               n_jobs = 8,
                               verbose = 3)
    
    
    grid_search.fit(X_train, y_train)
    return(grid_search)

### Logistic Regression without SMOTE

In [9]:
lr_model = lr_model_gscv(X_train, y_train, smote = False)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[IterativeImputer] Completing matrix with shape (36000, 185)
[IterativeImputer] Change: 4762.814312581376, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 902.9395589654421, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.


In [10]:
print('Best model parameters determine during grid search:', lr_model.best_params_)
print('Best AUC score on cross-validation data:', lr_model.best_score_)

Best model parameters determine during grid search: {'classifier__C': 0.1, 'classifier__max_iter': 1000}
Best AUC score on cross-validation data: 0.7659928781183799


In [11]:
pd.DataFrame(lr_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_classifier__max_iter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,106.492403,12.312691,0.635942,0.105717,0.01,1000,"{'classifier__C': 0.01, 'classifier__max_iter'...",0.760046,0.758299,0.769281,0.769006,0.770698,0.765466,0.005200,5
1,107.616596,11.507221,0.749675,0.071090,0.1,1000,"{'classifier__C': 0.1, 'classifier__max_iter':...",0.760664,0.759040,0.768502,0.769648,0.772109,0.765993,0.005173,1
2,108.832217,12.584224,0.713561,0.083899,1,1000,"{'classifier__C': 1, 'classifier__max_iter': 1...",0.760565,0.759048,0.768056,0.769493,0.772339,0.765900,0.005185,2
3,105.461898,11.924182,0.616970,0.093569,10,1000,"{'classifier__C': 10, 'classifier__max_iter': ...",0.760552,0.759010,0.768011,0.769428,0.772377,0.765876,0.005195,3
4,80.957402,22.523959,0.321170,0.112937,100,1000,"{'classifier__C': 100, 'classifier__max_iter':...",0.760553,0.759006,0.768003,0.769421,0.772386,0.765874,0.005196,4


### Logistic Regression with SMOTE

In [12]:
lr_model_smote = lr_model_gscv(X_train, y_train, smote = True)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[IterativeImputer] Completing matrix with shape (36000, 185)
[IterativeImputer] Change: 4762.814312581376, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 902.9395589654421, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.


C:\Users\akipp\anaconda3\envs\py38env\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [13]:
print('Best model parameters determine during grid search:', lr_model_smote.best_params_)
print('Best AUC score on cross-validation data:', lr_model_smote.best_score_)

Best model parameters determine during grid search: {'classifier__C': 100, 'classifier__max_iter': 1000}
Best AUC score on cross-validation data: 0.7656243496126398


In [14]:
pd.DataFrame(lr_model_smote.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_classifier__max_iter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,114.413260,11.337145,0.693018,0.043407,0.01,1000,"{'classifier__C': 0.01, 'classifier__max_iter'...",0.758626,0.757595,0.766974,0.768843,0.771542,0.764716,0.005595,5
1,115.908116,9.870981,0.734868,0.067531,0.1,1000,"{'classifier__C': 0.1, 'classifier__max_iter':...",0.760119,0.758592,0.767243,0.769535,0.772054,0.765509,0.005272,4
2,129.773434,10.790150,0.672348,0.144305,1,1000,"{'classifier__C': 1, 'classifier__max_iter': 1...",0.760452,0.758706,0.767045,0.769565,0.772027,0.765559,0.005160,3
3,146.885308,8.930459,0.552757,0.079529,10,1000,"{'classifier__C': 10, 'classifier__max_iter': ...",0.760588,0.758636,0.767198,0.769476,0.772137,0.765607,0.005175,2
4,120.952117,28.495526,0.295868,0.070144,100,1000,"{'classifier__C': 100, 'classifier__max_iter':...",0.760556,0.758790,0.767237,0.769321,0.772217,0.765624,0.005140,1


From the above results, it appears that while the classes are imbalanced, using SMOTE to aftifically balance the classes does not provide any particular benefit so it can be dropped from use in Logistic Regression.

### Logistic Regression Intermediate Test Data Score - Without SMOTE

In [15]:
print('AUC on test data:', lr_model.score(X_test, y_test))

[IterativeImputer] Completing matrix with shape (4000, 185)
AUC on test data: 0.7478387779794312


## Random Forest Modeling Pipeline

In [17]:
def cbc_model_gscv(X_train, y_train, smote = True):
    # One-Hot Enconde categorical features
    cat_feats = X_train.columns[X_train.dtypes == 'object']
    
    # An imblearn pipeline is needed when using SMOTE in the pipeline because the function doesn't 
    # have a fit_transform method and can't be used with a sklearn pipeline. Also, the imblearn 
    # pipeline will ensure that SMOTE is only applied to training data, even in cross-validation.
    if smote == True: 
        pipeline = imbpipeline(steps = [('ohe', ColumnTransformer(transformers = [('ohe', OneHotEncoder(handle_unknown = 'ignore', sparse = False), cat_feats)], remainder = 'passthrough')),
                                        # Inputing missing continuous feature data with chained equations
                                        ('mice', IterativeImputer(max_iter = 20, tol = 0.01, n_nearest_features = 50, random_state = 42, verbose = 1)), 
                                        # Creating synthetic data from the minority class
                                        ('smote', SMOTE(k_neighbors = 10, random_state = 42)), 
                                        # Attempting to normalize the features with the yeo-johnson transformation (box-cox also an option, but requires all data to be positive)
                                        ('gaussian', PowerTransformer()),
                                        # Robust scaling of the data, rather than Standard scaling, because it is less impacted by outliers
                                        ('scaler', RobustScaler()),
                                        ('classifier', CatBoostClassifier(task_type = 'GPU', devices = '0:1', verbose = 0))
                                       ])
    # When not using SMOTE, a sklearn pipeline works fine
    else:
        pipeline = Pipeline(steps = [('ohe', ColumnTransformer(transformers = [('ohe', OneHotEncoder(handle_unknown = 'ignore', sparse = False), cat_feats)], remainder = 'passthrough')),
                                     # Inputing missing continuous feature data with chained equations
                                     ('mice', IterativeImputer(max_iter = 20, tol = 0.01, n_nearest_features = 50, random_state = 42, verbose = 1)),
                                     # Attempting to normalize the features with the yeo-johnson transformation (box-cox also an option, but requires all data to be positive)
                                     ('gaussian', PowerTransformer()),
                                      # Robust scaling of the data, rather than Standard scaling, because it is less impacted by outliers
                                     ('scaler', RobustScaler()),
                                     ('classifier', CatBoostClassifier(task_type = 'GPU', devices = '0:1', verbose = 0))
                                    ])
    # Specify how the folds should be created in GridSearchCV
    stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=42)
    # Define the parameter grid for GridSearchCV
    param_grid = {'classifier__iterations': [1000]} # 'classifier__iterations': [1000], 'classifier__n_estimators': [100, 200, 300], 'classifier__max_depth': [10, 15, 20]
    grid_search = GridSearchCV(estimator = pipeline,
                               param_grid = param_grid,
                               scoring = 'roc_auc',
                               cv = stratified_kfold,
                               n_jobs = 1,
                               verbose = 1)
    
    
    grid_search.fit(X_train, y_train)
    return(grid_search)

### Random Forest Classifier without SMOTE

In [18]:
cbc_model = cbc_model_gscv(X_train, y_train, smote = False)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[IterativeImputer] Completing matrix with shape (28800, 185)
[IterativeImputer] Change: 4772.316650594342, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 955.7651025295002, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.


C:\Users\akipp\anaconda3\envs\py38env\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[IterativeImputer] Completing matrix with shape (7200, 185)
[IterativeImputer] Completing matrix with shape (28800, 185)
[IterativeImputer] Change: 4522.071574019754, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 1121.1487511084902, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 772.098752699685, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.


C:\Users\akipp\anaconda3\envs\py38env\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[IterativeImputer] Completing matrix with shape (7200, 185)
[IterativeImputer] Completing matrix with shape (28800, 185)
[IterativeImputer] Change: 4726.374474804047, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 1289.3303725232347, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 910.748209593336, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.


...

[IterativeImputer] Completing matrix with shape (7200, 185)
[IterativeImputer] Completing matrix with shape (28800, 185)
[IterativeImputer] Change: 5156.155298589655, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 1118.8323782043637, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 559.6982391417464, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.


C:\Users\akipp\anaconda3\envs\py38env\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[IterativeImputer] Completing matrix with shape (7200, 185)
[IterativeImputer] Completing matrix with shape (28800, 185)
[IterativeImputer] Change: 4853.875315268089, scaled tolerance: 1052.68939228 
[IterativeImputer] Change: 1400.5024189753487, scaled tolerance: 1052.68939228 
[IterativeImputer] Change: 537.7464642603584, scaled tolerance: 1052.68939228 
[IterativeImputer] Early stopping criterion reached.


C:\Users\akipp\anaconda3\envs\py38env\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[IterativeImputer] Completing matrix with shape (7200, 185)
[IterativeImputer] Completing matrix with shape (36000, 185)
[IterativeImputer] Change: 4762.814312581376, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 902.9395589654421, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.


In [19]:
print('Best model parameters determine during grid search:', cbc_model.best_params_)
print('Best AUC score on cross-validation data:', cbc_model.best_score_)

Best model parameters determine during grid search: {'classifier__iterations': 1000}
Best AUC score on cross-validation data: 0.8033473151007945


In [20]:
pd.DataFrame(cbc_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__iterations,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,72.228737,2.78123,0.248729,0.021224,1000,{'classifier__iterations': 1000},0.798605,0.802302,0.804163,0.805833,0.805834,0.803347,0.002706,1


### Cat Boost Classifier with SMOTE

In [20]:
cbc_model_smote = cbc_model_gscv(X_train, y_train, smote = True)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[IterativeImputer] Completing matrix with shape (28800, 185)
[IterativeImputer] Change: 4772.316650594342, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 955.7651025295002, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (7200, 185)
[IterativeImputer] Completing matrix with shape (28800, 185)
[IterativeImputer] Change: 4522.071574019754, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 1121.1487511084902, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 772.098752699685, scaled tolerance: 1054.43357829 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (7200, 185)
[IterativeImputer] Completing matrix with shape (28800, 185)
[IterativeImputer] Change: 4726.374474804047, scaled tolerance: 1054.43357829 
[IterativeImputer] Change: 1289.3303725232347, scaled tole

In [21]:
print('Best model parameters determine during grid search:', cbc_model_smote.best_params_)
print('Best AUC score on cross-validation data:', cbc_model_smote.best_score_)

Best model parameters determine during grid search: {'classifier__iterations': 1000}
Best AUC score on cross-validation data: 0.8041748444595009


In [22]:
pd.DataFrame(cbc_model_smote.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__iterations,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,75.724974,2.935447,0.204446,0.014762,1000,{'classifier__iterations': 1000},0.799441,0.802491,0.805185,0.804121,0.809636,0.804175,0.003349,1


From the above results, it appears that while the classes are imbalanced, using SMOTE to aftifically balance the classes does not provide much, if any, benefit so it can be dropped from use in Cat Boost Classifier.

### Cat Boost Classifier Intermediate Test Data Score - Without SMOTE

In [21]:
print('AUC on test data:', cbc_model.score(X_test, y_test))

[IterativeImputer] Completing matrix with shape (4000, 185)
AUC on test data: 0.7941142367412786


## Final Test Predictions

In [24]:
test_data_name = 'exercise_40_test.csv'

In [25]:
test_data = pd.read_csv(data_path + test_data_name)

In [26]:
X_test = data_cleaning(test_data)

In [27]:
lr_test_probs = pd.DataFrame(lr_model.predict_proba(X_test))
lr_test_probs.columns = ['prob_class_0', 'prob_class_1']
lr_test_probs.drop(columns = ['prob_class_0'], inplace = True)
cbc_test_probs = pd.DataFrame(cbc_model.predict_proba(X_test))
cbc_test_probs.columns = ['prob_class_0', 'prob_class_1']
cbc_test_probs.drop(columns = ['prob_class_0'], inplace = True)

[IterativeImputer] Completing matrix with shape (10000, 185)
[IterativeImputer] Completing matrix with shape (10000, 185)


In [28]:
lr_test_probs.to_csv(data_path + 'glmresults.csv', header = False, index = False)
cbc_test_probs.to_csv(data_path + 'nonglmresults.csv', header = False, index = False)